### 비모수적 방법
: 표본이 정규분포를 따르지 않는 경우에 사용하는 방법이다.

- Wilcoxon signed rank test
- Wilcoxon rank sum test
- Mann-Whitney U test 
- Kruskal-Wallis test
- Friedman test 



#### Wilcoxon signed rank test (>> 단일 t-검정)<br>(윌콕슨 부호 순위 검정)
: 표본의 중앙값이 알려진 중앙값과 차이가 있는지 검증하는 방법이다.<br>
각 측정값에 알려진 중앙값을 빼고 부호를 무시하고 절대값으로 순위를 구한 후, 그 다음 순위에 부호를 붙여 구한다.

- 귀무가설 : 표본의 중앙값은 특정 값과 차이가 없다.
- 대립가설 : 표본의 중앙값은 특정 값과 차이가 있다.

In [404]:
from scipy.stats import shapiro

data = [99, 100, 90, 94, 135, 108, 107, 111, 119, 104, 127, 109, 117, 105, 185]

print("정규성:", shapiro(data)[1])

정규성: 0.0015666796825826168


In [405]:
from numpy import array
from scipy.stats import wilcoxon

data2 = array(data)

print(wilcoxon(data2 - 108)[1])

0.6152572302512083


<br><br>

#### Wilcoxon signed rank test (>> 대응 T-검정)
: 동일한 사람 또는 물건에 대한 표본이 두 개인 경우에 순위 차이가 있는지 검증하는 방법이다.<br>
시간 전 후의 개념이기 때문에 독립된 두 집단일 필요가 없다.

- 귀무가설 : 전과 후의 순위 차이가 없다.
- 대립가설 : 전과 후의 순위 차이가 있다.

In [425]:
before = [8, 5, 7, 6, 7, 8, 6, 5, 8, 17]
after = [10, 9, 7, 8, 9, 10, 10, 8, 9, 18]

print("before 정규성:", shapiro(before)[1], "\nafter 정규성:", shapiro(after)[1])

before 정규성: 0.0004462270298972726 
after 정규성: 0.00045770188444294035


In [426]:
print(wilcoxon(before, after)[1])

0.0070583320485281004


<br><br>

#### Mann-Whitney U test (>> 독립 t-검정)<br>(맨 휘트니 U 검정)
: 표본이 정규성을 만족하지 못할 때 표본간 순위 차이가 있는지 검증하는 방법이다.

- 귀무가설 : 두 표본의 순위 차이가 없다.
- 대립가설 : 두 표본의 순위 차이가 있다.

In [416]:
a_group = [85, 87, 86, 88, 89, 85, 86, 87, 88, 89]
b_group = [94, 98, 97, 95, 96, 95, 98, 97, 98, 1]

print("a_group 정규성:", shapiro(a_group)[1], "\nb_group 정규성:", shapiro(b_group)[1])

a_group 정규성: 0.34113210439682007 
b_group 정규성: 3.0795149541518185e-07


In [420]:
from scipy.stats import mannwhitneyu

print(mannwhitneyu(a_group, b_group)[1])

0.0013572543223486374


<br><br>

In [417]:
np.mean(a_group)

87.0

In [418]:
np.mean(b_group)

86.9

In [419]:
from scipy.stats import ttest_ind

print(ttest_ind(a_group, b_group)[1])

0.9917748790116712


<br>

#### Wilcoxon rank sum test (>> 독립 t-검정)<br>(윌콕슨 순위 합 검정)

In [422]:
from scipy.stats import ranksums

print(ranksums(a_group, b_group)[1])

0.002496908915141548


<br><br>

#### Kruskal-Wallis test (>> 분산분석)<br>(크루스칼 왈리스 검정)
: 세 개 이상의 표본을 비교할 때 사용하는 검정 방법으로 표본이 정규성을 만족하지 못할 때<br>
평균 차이가 있는지가 아닌 중앙값 차이가 있는지 검증하는 방법이다.

- 귀무가설 : 표본들의 중앙값이 차이가 없다.
- 대립가설 : 표본들의 중앙값에 적어도 하나 차이가 있다.

In [515]:
import pandas as pd

file_path = 'data_kruskal.xlsx'
df = pd.read_excel(file_path)
df.head()

,group,value
0,a_group,1
1,a_group,2
2,a_group,4
3,a_group,5
4,a_group,50


In [516]:
a_group = df[df.group == 'a_group'].value
b_group = df[df.group == 'b_group'].value
c_group = df[df.group == 'c_group'].value

In [517]:
from scipy.stats import shapiro

print("a_group 정규성:", shapiro(a_group)[1], 
      "\nb_group 정규성:", shapiro(b_group)[1], 
      "\nc_group 정규성:", shapiro(c_group)[1])

a_group 정규성: 0.006666063331067562 
b_group 정규성: 0.9135605096817017 
c_group 정규성: 0.01818660832941532


In [518]:
from scipy.stats import kruskal

print(kruskal(a_group, b_group, c_group)[1])

0.7209979468055585


In [520]:
from statsmodels.sandbox.stats.multicomp import MultiComparison
import scipy.stats

comp = MultiComparison(df['value'], df['group'])

result = comp.allpairtest(scipy.stats.mannwhitneyu, method='bonf')
result[0]

group1,group2,stat,pval,pval_corr,reject
a_group,b_group,39.5,0.4824,1.0,False
a_group,c_group,29.5,0.1765,0.5296,False
b_group,c_group,36.0,0.362,1.0,False


<br><br>

In [519]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

model = ols('value ~ C(group)', df).fit()
anova_lm(model)

,df,sum_sq,mean_sq,F,PR(>F)
C(group),2.0,2.280081e+06,1.140040e+06,5.117402,0.01409
Residual,24.0,5.346653e+06,2.227772e+05,NaN,NaN


<br><br>

#### Friedman test (>> 분산분석)<br>(프리드먼 검정)

<br>

<br><br><br>

<br>

### 카이제곱검정
: 두 범주형 변수들 간에 비율 차이가 있는지 검증하는 방법<br>
(독립변수와 종속변수 모두 범주형)

- 독립성을 만족해야 한다.
- 빈도가 5 이하인 셀이 전체의 20% 이하여야 한다.

<br>

귀무가설 : 두 변수들 간에는 비율의 차이가 없다.<br>
대립가설 : 두 변수들 간에는 비율의 차이가 있다.<br>


In [102]:
import pandas as pd

file_path = 'data_heart.csv'
df = pd.read_csv(file_path)
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [103]:
df.sex = df.sex.replace({1:'male', 0:'female'})
df.target = df.target.replace({1:'1', 0:'0'})

In [109]:
df2 = pd.crosstab(df.sex, df.target)
df2

target,0,1
sex,,
female,24,72
male,114,93


In [110]:
from scipy.stats import chi2_contingency

print(chi2_contingency(df2)[1])

1.8767776216941503e-06


<br><br>

#### Fisher's Exact test(피셔의 정확 검정)
: 테이블이 2* 2 이고 빈도가 5보다 작은 셀이 20% 이상인 경우에 사용

In [85]:
file_path2 = 'data_fisher.xlsx'
df3 = pd.read_excel(file_path2)
df3.head()

,sex,method
0,male,b
1,female,b
2,female,a
3,male,a
4,female,b


In [86]:
df4 = pd.crosstab(df3.sex, df3.method)
df4

method,a,b
sex,,
female,4,20
male,3,23


In [87]:
from scipy.stats import fisher_exact

print(fisher_exact(df4)[1])

0.6970618034447817


<br><br>

#### McNemar's test(맥니마 검정)
: 테이블이 2* 2이고 시간 전 후의 측정 표본일 경우에 사용

In [119]:
file_path3 = 'data_mcnemar.xlsx'
df5 = pd.read_excel(file_path3)
df5.head()

,before,after
0,A,A
1,B,A
2,A,A
3,A,A
4,A,A


In [120]:
df6 = pd.crosstab(df5.before, df5.after)
df6

after,A,B
before,,
A,24,11
B,34,17


In [121]:
from statsmodels.sandbox.stats.runs import mcnemar

print(mcnemar(df6)[1])

0.0008240823595997424
